<a href="https://colab.research.google.com/github/Okocha76/DS-Unit-2-Applied-Modeling/blob/master/module4/LS_DS10_234_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Lambda School Data Science

*Unit 2, Sprint 3, Module 4*

---

# Model Interpretation

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make at least 1 partial dependence plot to explain your model.
- [ ] Make at least 1 Shapley force plot to explain an individual prediction.
- [ ] **Share at least 1 visualization (of any type) on Slack!**

If you aren't ready to make these plots with your own dataset, you can practice these objectives with any dataset you've worked with previously. Example solutions are available for Partial Dependence Plots with the Tanzania Waterpumps dataset, and Shapley force plots with the Titanic dataset. (These datasets are available in the data directory of this repository.)

Please be aware that **multi-class classification** will result in multiple Partial Dependence Plots (one for each class), and multiple sets of Shapley Values (one for each class).

## Stretch Goals

#### Partial Dependence Plots
- [ ] Make multiple PDPs with 1 feature in isolation.
- [ ] Make multiple PDPs with 2 features in interaction. 
- [ ] Use Plotly to make a 3D PDP.
- [ ] Make PDPs with categorical feature(s). Use Ordinal Encoder, outside of a pipeline, to encode your data first. If there is a natural ordering, then take the time to encode it that way, instead of random integers. Then use the encoded data with pdpbox. Get readable category names on your plot, instead of integer category codes.

#### Shap Values
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!

### Links

#### Partial Dependence Plots
- [Kaggle / Dan Becker: Machine Learning Explainability — Partial Dependence Plots](https://www.kaggle.com/dansbecker/partial-plots)
- [Christoph Molnar: Interpretable Machine Learning — Partial Dependence Plots](https://christophm.github.io/interpretable-ml-book/pdp.html) + [animated explanation](https://twitter.com/ChristophMolnar/status/1066398522608635904)
- [pdpbox repo](https://github.com/SauceCat/PDPbox) & [docs](https://pdpbox.readthedocs.io/en/latest/)
- [Plotly: 3D PDP example](https://plot.ly/scikit-learn/plot-partial-dependence/#partial-dependence-of-house-value-on-median-age-and-average-occupancy)

#### Shapley Values
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

## Data import & exploration

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pdpbox
    !pip install shap

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Okocha76/Okocha76.github.io/master/euro_data.csv', sep=';', engine='python', encoding = "ISO-8859-1")
df.shape

(8010, 41)

In [0]:
train = df[df['TIME'] <= 2011]
val = df[(df['TIME'] > 2011) & (df['TIME'] < 2015)]
test = df[df['TIME'] >= 2015]

train.shape, val.shape, test.shape

((5340, 41), (1335, 41), (1335, 41))

In [0]:
import numpy as np

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()

    # Feature selection is not final. Just a quick & dirty approach for now.

    X = X.drop(
        ['CNMIGRAT','DEATH','GROW','GROWRT','LBIRTH','NATGROW','NATGROWRT',
        'EUR_HAB_2GDP','EUR_HAB_EU27_2019','MIO_EUR','MIO_NAC','MIO_PPS',
        'MIO_PPS_EU27_2019','PPS_EU27_2019_HAB','PPS_HAB','PPS_HAB_EU27_2019',
        'SAL','EUR_HAB_2HHINC','ED0-2_25_64','ED3_4_25_64','ED3-8_25_64',
        'ED5-8_25_64','ED3-8_30_34','MEDAGEMOTH','PPS_HAB_EU','ED3_4_30_34']
        ,axis=1)
    
    X['JAN'] = pd.to_numeric(X['JAN'],errors='coerce')
    # X['JAN'] = X['JAN'].astype(int)
    X['TIME'] = X['TIME'].astype('category')

    # A bit of feature engineering. EMP_PC is percentage employed.
    X['EMP_PC'] = round((X['EMP'] * 100000)/X['JAN'],1)
    X['EMP_PC'] = X['EMP_PC'].replace(np.inf, np.nan)
    X = X.drop(['JAN','EMP'], axis=1)

    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    cols_with_zeros = ['AGEMOTH', 'ED0-2_30_34', 'ED5-8_30_34','EMP_PC',
                       'EUR_HAB_EU','GBIRTHRT','GDEATHRT','PER_KM2','RT',
                       'TOTFERRT']

    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)

    X = X.rename(columns={"ED0-2_30_34": "ED_LOW", "ED5-8_30_34": "ED_HIGH"})    

    # Make target feature categorical
    X['MIG_CAT'] = pd.cut(X['CNMIGRATRT'], [-np.inf, 0, 5, np.inf], labels=['0_low', '1_avg', '2_high']).astype(np.object)
    X = X.drop(['CNMIGRATRT'], axis=1)

    # Convert sub_grade from string "A1"-"D5" to numbers
    mig_rates = {'0_low': 1, '1_avg': 2, '2_high': 3}
    X['MIG_CAT'] = X['MIG_CAT'].map(mig_rates)

    # return the wrangled dataframe
    return X


train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
train.sample(10)

,TIME,GEO,GEO_LABEL,AGEMOTH,TOTFERRT,GBIRTHRT,GDEATHRT,RT,PER_KM2,EUR_HAB_EU,ED_LOW,ED_HIGH,EMP_PC,MIG_CAT
6017,2005,SE2,Södra Sverige,30.3,1.76,11.0,10.3,2.5,51.2,135.0,7.7,34.9,47.5,2
4092,2006,IT,Italy,31.0,1.37,9.6,9.6,3.2,197.0,107.0,35.9,17.6,43.0,2
7372,2010,UKF3,Lincolnshire,28.5,1.94,10.9,10.3,3.7,120.1,80.0,16.6,28.2,41.1,3
7543,2001,UKI3,Inner London - West,NaN,NaN,NaN,NaN,NaN,NaN,676.0,NaN,NaN,NaN,1
7097,2005,UKC2,Northumberland and Tyne and Wear,28.4,1.65,10.7,10.8,3.7,251.4,116.0,24.5,27.7,45.8,2
184,2004,AT32,Salzburg,28.9,1.43,10.2,7.7,4.8,73.9,149.0,11.5,18.3,52.2,2
2094,2006,EL3,Attiki,31.0,1.31,10.4,8.7,3.8,1044.9,106.0,17.7,32.1,45.8,2
370,2010,BE24,Prov. Vlaams-Brabant,30.7,1.78,10.8,9.0,3.3,516.6,138.0,12.3,55.9,40.0,3
2798,2008,ES62,Región de Murcia,30.1,1.70,13.6,7.3,3.4,126.7,78.0,46.9,32.3,44.9,3
7206,2006,UKD7,Merseyside,28.8,1.78,11.7,10.5,5.5,2057.1,111.0,25.9,31.9,40.9,1


In [0]:
type(train['MIG_CAT'])

pandas.core.series.Series

## Partial Dependence Plot

In [0]:
# Later, when you save matplotlib images to include in blog posts or web apps,
# increase the dots per inch (double it), so the text isn't so fuzzy
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 72

In [0]:
target = 'MIG_CAT'
features = train.columns.drop([target, 'TIME', 'GEO_LABEL'])

train_pdp = train.dropna()
val_pdp = val.dropna()
test_pdp = test.dropna()

X_train = train_pdp[features]
X_val = val_pdp[features]
X_test = test_pdp[features]

y_train = train_pdp[target]
y_val = val_pdp[target]
y_test = test_pdp[target]

In [0]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline


pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))

Validation Accuracy 0.5760299625468165


In [0]:
from pdpbox.pdp import pdp_isolate, pdp_plot

feature = 'ED_LOW'

isolated = pdp_isolate(
    model=pipeline, 
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=feature
)

In [0]:
plt.rcParams['figure.dpi'] = 100

pdp_plot(isolated, 
         feature_name=feature, 
        #  x_quantile=True, 
         plot_pts_dist=True,
         ncols=1,         
         plot_params = {
                     # plot title and subtitle
                    'title': 'PDP for feature ED_LOW (Percentage of population with less than primary, primary and lower secondary education)', 
                    'title_fontsize': 14,                   
                    'subtitle': 'class 0: Migration Rate <= 0 %, class 1: 0 % < Migration Rate <= 5 %, class 2: Migration Rate > 5 %',
                    'subtitle_fontsize': 14                 
                }
         );

## Shapley

In [0]:
# Arrange data into X features matrix and y target vector
target = 'MIG_CAT'
features = train.columns.drop([target, 'TIME', 'GEO_LABEL'])

X_train = train[features]
X_val = val[features]
X_test = test[features]

y_train = train[target]
y_val = val[target]
y_test = test[target]

In [0]:
from xgboost import XGBClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

processor = make_pipeline(
    ce.OrdinalEncoder()
)

X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

model = XGBClassifier(n_estimators=1000, n_jobs=-1)
model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=10)

XGBoostError: ignored

In [0]:
row = X_test.iloc[[0]]
row

,GEO,AGEMOTH,TOTFERRT,GBIRTHRT,GDEATHRT,RT,PER_KM2,EUR_HAB_EU,ED_LOW,ED_HIGH,EMP_PC
15,AT,30.6,1.49,9.8,9.6,3.1,104.9,137.0,11.1,38.7,49.9


In [0]:
y_test.iloc[[0]]

15    2_high
Name: MIG_CAT, dtype: object

In [0]:
# STUDY/PRACTICE THIS CELL FOR THE SPRINT CHALLENGE
import shap

explainer = shap.TreeExplainer(model)
row_processed = processor.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row, 
    link='logit' # For classification, this shows predicted probabilities
)

AttributeError: ignored